In [1]:
from eventregistry import *
import json, os, sys

from dotenv import load_dotenv
import os
from eventregistry import EventRegistry


load_dotenv()  # This loads the environment variables from .env file
api_key = os.getenv("EVENT_REGISTRY_API_KEY")
er = EventRegistry(apiKey=api_key, allowUseOfArchive=False)

In [6]:
conceptUri = "Territorial_disputes_in_the_South_China_Sea"
lang = "eng"
query = {
    "$query": {
        "conceptUri": "http://en.wikipedia.org/wiki/Territorial_disputes_in_the_South_China_Sea",
        "lang": lang,
        "isDuplicate": False,
    },
}

q = QueryArticlesIter.initWithComplexQuery(query)

filename = lang + "_" + conceptUri + ".json"
if os.path.exists(filename):
    os.remove(filename)

articles = []
for art in q.execQuery(
    er,
    sortBy="date",
    maxItems=100,
    returnInfo=ReturnInfo(
        sourceInfo=SourceInfoFlags(location=True),
    ),
):
    articles.append(art)

with open(filename, "w") as f:
    json.dump(articles, f, ensure_ascii=False, indent=4)

In [8]:
# copy the body of the articles to a new json file
"""
import json

articles_body = []
for article in articles:
    art = {}
    art["article"] = article["body"]
    articles_body.append(art)
print(articles_body)
"""

[{'article': 'Ukrainian President Volodymyr Zelenskyy met his Philippine counterpart, President Ferdinand Marcos Jr., in Manila on Monday, a day after the conclusion of a security summit where the two leaders delivered statements sharply critical of China.\n\nThe two leaders discussed a Ukraine summit which will be held in Switzerland June 15-16, and Zelenskyy underscored the importance for Southeast Asian countries to be represented there.\n\n"I am pleased that the Philippines will participate in the summit," he said in a post on X. Marcos\' communications office confirmed that the Philippines is joining the event, but is still finalizing details.\n\n"We will continue to do all that we can to promote peace and to bring an end to the fighting and to come to a political resolution of your country," Marcos told Zelenskyy during their meeting, according to the Philippine president\'s communications office.\n\nThe Ukraine leader\'s visit to the Philippines followed his attendance at the Sh

In [9]:
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [18]:
def QnA(
    art,
    client,
    model="gpt-4o",
    QnA_prompt="I have 10 news articles, and I need you to create multiple-choice questions with categorical answers to identify narrative differences among these articles. Here are the articles:",
):

    prompt_complete = QnA_prompt + "\n" + art
    model = model
    messages = [
        {"role": "user", "content": prompt_complete},
    ]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [19]:
response = QnA(str(articles_body), client)

In [20]:
print(response)

Here are some multiple-choice questions with categorical answers to identify narrative differences among the provided articles:

### Question 1: What is the primary focus of the meeting between Ukrainian President Volodymyr Zelenskyy and Philippine President Ferdinand Marcos Jr.?
A. Strengthening military alliances  
B. Discussing a Ukraine summit in Switzerland  
C. Criticizing China's actions in the South China Sea  
D. Promoting economic cooperation

### Question 2: How does the article describe China's response to the Philippines' actions in the South China Sea?
A. China seeks dialogue to handle maritime conflicts  
B. China plans to increase military presence in the region  
C. China will take legal action against the Philippines  
D. China will impose economic sanctions on the Philippines

### Question 3: What is a common theme in the articles regarding the Shangri-La Dialogue?
A. Economic cooperation between countries  
B. Criticism of China's actions in the South China Sea  
C.

In [36]:
import json


def get_LLM_QnA(
    src_filename,
    client,
    model="gpt-4o",
    QnA_prompt="Which of the following is correct tone that this article discuss about Raisi's reputation:",
    QnA_answer="A) Positive\nB) Neutral\nC) Negative\nD) Not applicable\n",
):
    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)
        for src_art in src_articles:
            src_art["Question"] = QnA_prompt
            src_art["Answers"] = QnA_answer
            src_art["LLM_answer"] = QnA(
                src_art,
                client,
                model=model,
                QnA_prompt=QnA_prompt,
                QnA_answer=QnA_answer,
            )

            des_articles.append(src_art)
    des_filename = "QnA_" + src_filename
    if os.path.exists(des_filename):
        os.remove(des_filename)
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

## Test question for article interest

In [121]:
question = "What approach does the article prefer for resolving the territorial disputes in the South China Sea:"
answer = "A) Bilateral negotiations \
B) Multilateral talks  \
C) International law  \
D) Economic cooperation and maintaining stability"
get_LLM_QnA(
    "Territorial_disputes_in_the_South_China_Sea.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
    QnA_answer=answer,
)

In [45]:
import re


def extract_answer(qna_text):
    # Use a regular expression to find the answer choice (A, B, C, or D) at the start of the QnA field
    match = re.search(r"\s*([A-D])\)", qna_text)
    if match:
        return match.group(1)
    return None


def extract_answer_from_json(src_filename):
    # Load the JSON data
    with open(src_filename, "r") as file:
        data = json.load(file)

    # Iterate through the records and extract answers
    for record in data:
        LLM_answer = record.get("LLM_answer", "")
        answer = extract_answer(LLM_answer)
        if answer:
            print(f"Extracted answer: {answer}, lang: {record['lang']}")
        else:
            print("No answer extracted")

In [122]:
extract_answer_from_json("QnA_Territorial_disputes_in_the_South_China_Sea.json")

Extracted answer: B, lang: eng
Extracted answer: B, lang: zho
Extracted answer: C, lang: zho
Extracted answer: A, lang: zho
Extracted answer: A, lang: zho
Extracted answer: B, lang: eng
Extracted answer: B, lang: eng
Extracted answer: B, lang: eng
Extracted answer: B, lang: eng
Extracted answer: B, lang: eng


In [38]:
def copy_LLM_QnA(
    src_filename,
    des_filename,
    N=10,
):
    with open(des_filename, "r") as f:
        data = f.read()
        des_articles = json.loads(data)

    with open(src_filename, "r") as f:
        data = f.read()

        src_articles = json.loads(data)
        for n in range(N):
            qna = {}
            qna["Question"] = src_articles[n]["Question"]
            qna["Answers"] = src_articles[n]["Answers"]
            qna["LLM_answer"] = src_articles[n]["LLM_answer"]
            match = re.search(r"\s*([A-D])\)", qna["LLM_answer"])
            if match:
                qna["post_LLM_answer"] = match.group(1)
            else:
                qna["post_LLM_answer"] = "NaN"

            if "QnA" in des_articles[n]:
                des_articles[n]["QnA"].append(qna)
            else:
                des_articles[n]["QnA"] = [qna]

            # overwrite the file
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [123]:
copy_LLM_QnA(
    src_filename="QnA_Territorial_disputes_in_the_South_China_Sea.json",
    des_filename="Interest_Territorial_disputes_in_the_South_China_Sea.json",
    N=10,
)

## Test different questions for article topical alignment

In [115]:
question = "Does the article discuss bilateral and multilateral negotiations:"
answer = "A) Yes \
B) No  "
get_LLM_QnA(
    "Territorial_disputes_in_the_South_China_Sea.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
    QnA_answer=answer,
)

In [116]:
extract_answer_from_json("QnA_Territorial_disputes_in_the_South_China_Sea.json")

Extracted answer: A, lang: eng
Extracted answer: A, lang: zho
Extracted answer: B, lang: zho
Extracted answer: B, lang: zho
Extracted answer: A, lang: zho
Extracted answer: A, lang: eng
Extracted answer: A, lang: eng
Extracted answer: A, lang: eng
Extracted answer: A, lang: eng
Extracted answer: A, lang: eng


In [117]:
copy_LLM_QnA(
    src_filename="QnA_Territorial_disputes_in_the_South_China_Sea.json",
    des_filename="Alignment_Territorial_disputes_in_the_South_China_Sea.json",
    N=10,
)